#  Neural Machine Translation

This translation model has been heavily inspired from coursera, Deeplearing.ai, course on Deep learning specialisation.

The included utility file, nmt_utils ha been include as is as provide in the coourse
 
Neural Machine Translation (NMT) model to translate human readable dates ("25th of June, 2009") into machine readable dates ("2009-06-25"). You will do this using an attention model, one of the most sophisticated sequence to sequence models. 
requires keras

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Dataset

We will train the model on a dataset of 10000 human readable dates and their equivalent, standardized, machine readable dates. Let's run the following cells to load the dataset and print some examples. 

In [2]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 27841.05it/s]


In [3]:
dataset[:10]
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

we've loaded:
- `dataset`: a list of tuples of (human readable date, machine readable date)
- `human_vocab`: a python dictionary mapping all characters used in the human readable dates to an integer-valued index 
- `machine_vocab`: a python dictionary mapping all characters used in machine readable dates to an integer-valued index. These indices are not necessarily consistent with `human_vocab`. 
- `inv_machine_vocab`: the inverse dictionary of `machine_vocab`, mapping from indices back to characters. 

Let's preprocess the data and map the raw text data into the index values. We will also use Tx=30 (which we assume is the maximum length of the human readable date; if we get a longer input, we would have to truncate it) and Ty=10 (since "YYYY-MM-DD" is 10 characters long). 

In [4]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


You now have:
- `X`: a processed version of the human readable dates in the training set, where each character is replaced by an index mapped to the character via `human_vocab`. Each date is further padded to $T_x$ values with a special character (< pad >). `X.shape = (m, Tx)`
- `Y`: a processed version of the machine readable dates in the training set, where each character is replaced by the index it is mapped to in `machine_vocab`. You should have `Y.shape = (m, Ty)`. 
- `Xoh`: one-hot version of `X`, the "1" entry's index is mapped to the character thanks to `human_vocab`. `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh`: one-hot version of `Y`, the "1" entry's index is mapped to the character thanks to `machine_vocab`. `Yoh.shape = (m, Tx, len(machine_vocab))`. Here, `len(machine_vocab) = 11` since there are 11 characters ('-' as well as 0-9). 


Lets also look at some examples of preprocessed training examples. 

In [5]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#description of the attention model has been provided int the root of the repository


In [6]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [46]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
   
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [8]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [50]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
   
    # Step 1: Define your pre-attention Bi-LSTM
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        
        context = one_step_attention(a, s)
        
        s, _, c = post_activation_LSTM_cell(initial_state = [s, c], inputs = context)
        out = output_layer(s)
        outputs.append(out)
    
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

Run the following cell to create your model.

In [51]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

Let's get a summary of the model to check if it matches the expected output.

#create an optimizer object

In [53]:

opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt,
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')

defining inputs and compiling the model

In [14]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

Let's now fit the model and run it for one epoch.

In [42]:
model.fit([Xoh, s0, c0], outputs, epochs=100, batch_size=100)

Epoch 1/100
10000/10000 [==============================] - 5s 459us/step - loss: 7.6239 - dense_3_loss: 2.0144 - dense_3_acc: 0.9732 - dense_3_acc_1: 0.9774 - dense_3_acc_2: 0.6016 - dense_3_acc_3: 0.2948 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9462 - dense_3_acc_6: 0.5247 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.5646 - dense_3_acc_9: 0.2539
Epoch 2/100
10000/10000 [==============================] - 5s 467us/step - loss: 6.7051 - dense_3_loss: 1.8549 - dense_3_acc: 0.9791 - dense_3_acc_1: 0.9796 - dense_3_acc_2: 0.6425 - dense_3_acc_3: 0.3791 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9616 - dense_3_acc_6: 0.6247 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.6327 - dense_3_acc_9: 0.3056
Epoch 3/100
10000/10000 [==============================] - 5s 467us/step - loss: 5.9269 - dense_3_loss: 1.6735 - dense_3_acc: 0.9798 - dense_3_acc_1: 0.9816 - dense_3_acc_2: 0.6963 - dense_3_acc_3: 0.4430 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9687 - dense_3_acc_6: 0.7123 - dense_3_acc_7: 0.9999 -

10000/10000 [==============================] - 5s 523us/step - loss: 1.5051 - dense_3_loss: 0.3676 - dense_3_acc: 0.9964 - dense_3_acc_1: 0.9962 - dense_3_acc_2: 0.9099 - dense_3_acc_3: 0.9457 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9889 - dense_3_acc_6: 0.9126 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.8813 - dense_3_acc_9: 0.8800
Epoch 25/100
10000/10000 [==============================] - 5s 505us/step - loss: 1.4636 - dense_3_loss: 0.3593 - dense_3_acc: 0.9963 - dense_3_acc_1: 0.9963 - dense_3_acc_2: 0.9108 - dense_3_acc_3: 0.9478 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9881 - dense_3_acc_6: 0.9181 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.8842 - dense_3_acc_9: 0.8815
Epoch 26/100
10000/10000 [==============================] - 5s 488us/step - loss: 1.4186 - dense_3_loss: 0.3497 - dense_3_acc: 0.9973 - dense_3_acc_1: 0.9970 - dense_3_acc_2: 0.9141 - dense_3_acc_3: 0.9465 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9888 - dense_3_acc_6: 0.9195 - dense_3_acc_7: 1.0000 - dense_3_a

10000/10000 [==============================] - 5s 548us/step - loss: 0.9190 - dense_3_loss: 0.2467 - dense_3_acc: 0.9990 - dense_3_acc_1: 0.9999 - dense_3_acc_2: 0.9636 - dense_3_acc_3: 0.9596 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9913 - dense_3_acc_6: 0.9510 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9433 - dense_3_acc_9: 0.9106
Epoch 48/100
10000/10000 [==============================] - 5s 502us/step - loss: 0.9040 - dense_3_loss: 0.2443 - dense_3_acc: 0.9993 - dense_3_acc_1: 0.9999 - dense_3_acc_2: 0.9658 - dense_3_acc_3: 0.9622 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9916 - dense_3_acc_6: 0.9504 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9449 - dense_3_acc_9: 0.9107
Epoch 49/100
10000/10000 [==============================] - 5s 492us/step - loss: 0.8887 - dense_3_loss: 0.2408 - dense_3_acc: 0.9992 - dense_3_acc_1: 0.9999 - dense_3_acc_2: 0.9675 - dense_3_acc_3: 0.9611 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9911 - dense_3_acc_6: 0.9518 - dense_3_acc_7: 1.0000 - dense_3_a

10000/10000 [==============================] - 5s 508us/step - loss: 0.6464 - dense_3_loss: 0.1876 - dense_3_acc: 0.9998 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9911 - dense_3_acc_3: 0.9892 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9925 - dense_3_acc_6: 0.9625 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9677 - dense_3_acc_9: 0.9376
Epoch 71/100
10000/10000 [==============================] - 5s 487us/step - loss: 0.6382 - dense_3_loss: 0.1859 - dense_3_acc: 0.9998 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9927 - dense_3_acc_3: 0.9895 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9926 - dense_3_acc_6: 0.9638 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9679 - dense_3_acc_9: 0.9387
Epoch 72/100
10000/10000 [==============================] - 5s 489us/step - loss: 0.6284 - dense_3_loss: 0.1837 - dense_3_acc: 0.9997 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9924 - dense_3_acc_3: 0.9904 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9929 - dense_3_acc_6: 0.9626 - dense_3_acc_7: 1.0000 - dense_3_a

10000/10000 [==============================] - 5s 477us/step - loss: 0.4811 - dense_3_loss: 0.1455 - dense_3_acc: 0.9999 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9980 - dense_3_acc_3: 0.9974 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9933 - dense_3_acc_6: 0.9674 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9784 - dense_3_acc_9: 0.9572
Epoch 94/100
10000/10000 [==============================] - 5s 516us/step - loss: 0.4770 - dense_3_loss: 0.1448 - dense_3_acc: 0.9999 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9982 - dense_3_acc_3: 0.9977 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9931 - dense_3_acc_6: 0.9692 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9790 - dense_3_acc_9: 0.9562
Epoch 95/100
10000/10000 [==============================] - 5s 471us/step - loss: 0.4698 - dense_3_loss: 0.1421 - dense_3_acc: 0.9999 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9979 - dense_3_acc_3: 0.9979 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9934 - dense_3_acc_6: 0.9687 - dense_3_acc_7: 1.0000 - dense_3_a

In [43]:
Xoh.shape
Xoh[1].shape

(30, 37)

In [44]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.transpose(source).reshape(1, 30,37)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1979-05-03
source: 5 April 09
output: 2009-04-04
source: 21th of August 2016
output: 2016-08-20
source: Tue 10 Jul 2007
output: 2007-07-10
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2001-03-03
source: March 3rd 2001
output: 2001-03-03
source: 1 March 2001
output: 2001-03-01


In [45]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          